# Agentic AI MCP - Client Only

This notebook connects to a remote MCP server and runs agentic workflows.
The MCP server should be running on another machine (see `server.ipynb`).

This machine only needs the Anthropic API key to run the LLM.

## Step 1: Setup

In [1]:
# Install if needed
# !uv pip install agentic-ai-mcp==0.4.2

import agentic_ai_mcp
print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.4.2


## Step 2: Make sure you have ANTHROPIC_API_KEY

The LLM runs on this machine, so you need the API key here.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("ANTHROPIC_API_KEY"):
    print("ANTHROPIC_API_KEY is set")
else:
    print("WARNING: ANTHROPIC_API_KEY not found. Set it in your .env file.")

ANTHROPIC_API_KEY is set


## Step 3: Connect to Remote MCP Server

Use the `mcp_url` parameter to connect to an existing MCP server.

In [7]:
from agentic_ai_mcp import AgenticAI

# Connect to remote MCP server (change this to your server's URL)
MCP_SERVER_URL = "http://127.0.0.1:8888/mcp"  # Change to remote IP if needed

ai = AgenticAI(
    mcp_url=MCP_SERVER_URL,  # Client-only mode: connect to existing server
    verbose=False
)

print(f"Client configured to connect to: {MCP_SERVER_URL}")

Client configured to connect to: http://127.0.0.1:8888/mcp


## Step 4: Run Simple Agent Task

In [8]:
result = await ai.run("Calculate 10 + 20, then multiply the result by 3")
print(result)

The answer is **90**.

Here's the breakdown:
- 10 + 20 = 30
- 30 × 3 = 90


## Step 5: Run Complex Task with Planning

In [9]:
result = await ai.run_with_planning("First calculate ((1+2)+(1+1))+1, then greet 'Alice' with the result number of times")
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Final Response

I have successfully completed the multi-step task:

**Calculation:** ((1+2)+(1+1))+1 = (3+2)+1 = 5+1 = **6**

**Greeting Result:** Alice has been greeted **6 times**:

> Hello, Alice! Hello, Alice! Hello, Alice! Hello, Alice! Hello, Alice! Hello, Alice!
